# Regularization
---
Regularization helps to prevent model from overfitting by adding an extra penelization term at the end of the loss function.

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$
To:
$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} \tag{2}$$

Where $m$ is the batch size. The shown regularization is called `L2 regularization`, where `L2` applies square to weights, `L1 regularization` applies absolute value, which has the form of $|W|$.

The appended extra term would enlarge the loss when either there are too many weights or the weight becomes too large, and the adjustable factor $\lambda$ emphasis on how much we want to penalize on the weights.

_**1. Why penalizing weights would help to prevent overfitting?**_

An intuitive understanding would be that in the process of minimizing the new loss function, some of the weights would decrease close to zero so that the corresponding neurons would have very small effect to our results, as if we are training on a smaller neural network with fewer neurons.

# Forward
---
In the forward process, we need only to change the loss function. let's review the cost function we've built in `deepNN`.

In [1]:
import numpy as np
from model import deepNN

In [2]:
model = deepNN([2, 4, 1])

In [3]:
A = np.array([[.3, .5, .7]])
Y = np.array([[1, 1, 1]])

loss = model.compute_cost(A, Y)
print(f'loss: {loss}')

loss: 0.7512649762748712


In [13]:
def compute_loss(A, Y, parameters, reg=True, lambd=.2):
    """
    With L2 regularization
    parameters: dict with 'W1', 'b1', 'W2', ...
    """
    assert A.shape == Y.shape
    n_layer = len(parameters)//2
    m = A.shape[1]
    s = np.dot(Y, np.log(A.T)) + np.dot(1-Y, np.log((1 - A).T))
    loss = -s/m
    if reg:
        p = 0
        for i in range(1, n_layer+1):
            p += np.sum(np.square(parameters['W'+str(i)]))
        loss += (1/m)*(lambd/2)*p
    return np.squeeze(loss)

In [6]:
model.weights_init()
model.params

{'W1': array([[ 0.00224882, -0.00683036],
        [-0.0155842 ,  0.00439355],
        [ 0.0026745 ,  0.00287223],
        [-0.00977243,  0.00515391]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[-0.02002206,  0.00227708,  0.00470624,  0.00502016]]),
 'b2': array([[0.]])}

In [14]:
loss = compute_loss(A, Y, model.params)
print(f'loss: {loss}')

loss: 0.7512951351356093


# Backward
---
The backward propagation of `L2 reglularization` is actually straight forward, we only need to add the gradient of the L2 term.

$$ \underbrace{\frac{\partial{J}^{\text{L2 Reg}}}{\partial{W}}}_{\text{new gradient}} = \underbrace{ \frac{\partial{J}^{\text{old}}}{\partial{W}} }_{\text{new gradient}} + \frac{\lambda}{m}|W|$$

In [15]:
def backward(params, cache, X, Y, lambd=0.2):
    """
    params: weight [W, b]
    cache: result [A, Z]
    Y: shape (1, m)
    """
    grad = {}
    n_layers = int(len(params)/2)
    m = Y.shape[1]
    cache['A0'] = X
    
    for l in range(n_layers, 0, -1):
        A, A_prev, Z = cache['A' + str(l)], cache['A' + str(l-1)], cache['Z' + str(l)]
        W = params['W'+str(l)]
        if l == n_layers:
            dA = -np.divide(Y, A) + np.divide(1 - Y, 1 - A)
        
        if l == n_layers:
            dZ = np.multiply(dA, sigmoid_grad(A, Z))
        else:
            dZ = np.multiply(dA, relu_grad(A, Z))
        
        # with an extra gradient at the end, other terms would remain the same
        dW = np.dot(dZ, A_prev.T)/m + (lambd/m)*W
        
        db = np.sum(dZ, axis=1, keepdims=True)/m
        dA = np.dot(W.T, dZ)

        grad['dW'+str(l)] = dW
        grad['db'+str(l)] = db
    
    return grad

# Ensemble

In [36]:
class deepNN:
    def __init__(self, layers):
        self.layers = layers
        self.params = {}
        self.reg = False
        self.lambd = .2
       
    
    def weights_init(self):
        n = len(self.layers)
        for i in range(1, n):
            self.params['W' + str(i)] = np.random.randn(self.layers[i], self.layers[i-1])*0.01
            self.params['b' + str(i)] = np.zeros((self.layers[i], 1))
    
    @staticmethod
    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    @staticmethod
    def relu(x):
        return np.maximum(x, 0)
    
    def compute_loss(self, A, Y):
        """
        With L2 regularization
        """
        assert A.shape == Y.shape
        n_layer = len(self.params)//2
        m = A.shape[1]
        s = np.dot(Y, np.log(A.T)) + np.dot(1-Y, np.log((1 - A).T))
        loss = -s/m
        if self.reg:
            p = 0
            for i in range(1, n_layer+1):
                p += np.sum(np.square(self.params['W'+str(i)]))
            loss += (1/m)*(self.lambd/2)*p
        return np.squeeze(loss)
    
    @staticmethod
    def sigmoid_grad(A, Z):
        grad = np.multiply(A, 1-A)
        return grad

    @staticmethod
    def relu_grad(A, Z):
        grad = np.zeros(Z.shape)
        grad[Z>0] = 1
        return grad
    
    
    def forward(self, X):
        # intermediate layer use relu as activation
        # last layer use sigmoid
        n_layers = int(len(self.params)/2)
        A = X
        cache = {}
        for i in range(1, n_layers):
            W, b = self.params['W'+str(i)], self.params['b'+str(i)]
            Z = np.dot(W, A) + b
            A = self.relu(Z)
            cache['Z'+str(i)] = Z
            cache['A'+str(i)] = A

        # last layer
        W, b = self.params['W'+str(i+1)], self.params['b'+str(i+1)]
        Z = np.dot(W, A) + b
        A = self.sigmoid(Z)
        cache['Z'+str(i+1)] = Z
        cache['A'+str(i+1)] = A

        return cache, A
    
    def backward(self, cache, X, Y):
        """
        cache: result [A, Z]
        Y: shape (1, m)
        """
        grad = {}
        n_layers = int(len(self.params)/2)
        m = Y.shape[1]
        cache['A0'] = X

        for l in range(n_layers, 0, -1):
            A, A_prev, Z = cache['A' + str(l)], cache['A' + str(l-1)], cache['Z' + str(l)]
            W = self.params['W'+str(l)]
            if l == n_layers:
                dA = -np.divide(Y, A) + np.divide(1 - Y, 1 - A)

            if l == n_layers:
                dZ = np.multiply(dA, self.sigmoid_grad(A, Z))
            else:
                dZ = np.multiply(dA, self.relu_grad(A, Z))
                
            dW = np.dot(dZ, A_prev.T)/m + (self.lambd/m)*W
            db = np.sum(dZ, axis=1, keepdims=True)/m
            dA = np.dot(W.T, dZ)

            grad['dW'+str(l)] = dW
            grad['db'+str(l)] = db

        return grad
    
    def optimize(self, grads, lr):
        n_layers = int(len(self.params)/2)
        for i in range(1, n_layers+1):
            dW, db = grads['dW'+str(i)], grads['db'+str(i)]
            self.params['W'+str(i)] -= lr*dW
            self.params['b'+str(i)] -= lr*db
    
    @staticmethod
    def generate_batch(X, batch_size):
        n = X.shape[0]
        batches = [range(i, i+batch_size) for i in range(0, n, batch_size)]
        return batches
    
    
    def train(self, X_train, y_train, batch_size=200, n_iter=100, lr=0.1, reg=True, lambd=.7):
        self.lambd = lambd
        self.reg = reg
        # prepare batch training
        batches = self.generate_batch(X_train, batch_size)
        # init weights
        self.weights_init()
        for i in range(n_iter):
            for batch in batches:
                X = X_train[batch, :].T
                Y = y_train[batch].reshape(1, -1)
                cache, A = self.forward(X)
                grads = self.backward(cache, X, Y)
                self.optimize(grads, lr)

            if i%10 == 0:
                loss = self.compute_loss(A, Y)
                print(f'iteration {i}: loss {loss}')

In [39]:
def accuracy(Y, Y_pred):
    """
    Y: vector of true value
    Y_pred: vector of predicted value
    """
    
    assert Y.shape[0] == 1
    assert Y.shape == Y_pred.shape
    Y_pred = np.round(Y_pred)
    acc = float(np.dot(Y, Y_pred.T) + np.dot(1 - Y, 1 - Y_pred.T))/Y.size
    return acc

In [17]:
from sklearn import datasets


X, y = datasets.make_classification(n_samples=10000, n_features=200, random_state=123)

X_train, X_test = X[:8000], X[8000:]
y_train, y_test = y[:8000], y[8000:]

print('train shape', X_train.shape)
print('test shape', X_test.shape)

train shape (8000, 200)
test shape (2000, 200)


In [37]:
layers = [200, 100, 20, 1]
model = deepNN(layers)

In [54]:
model.train(X_train, y_train, batch_size=200, n_iter=150, lr=0.05, reg=True, lambd=1)

iteration 0: loss 0.6985036506635248
iteration 10: loss 0.6974289293126693
iteration 20: loss 0.696563499398262
iteration 30: loss 0.6955727021117409
iteration 40: loss 0.6845170595754049
iteration 50: loss 0.23561800014771372
iteration 60: loss 0.15567224031891935
iteration 70: loss 0.12669228589646375
iteration 80: loss 0.11069865608869393
iteration 90: loss 0.1007637548980789
iteration 100: loss 0.09435682482867866
iteration 110: loss 0.09060941295356366
iteration 120: loss 0.08884491050012915
iteration 130: loss 0.08739359237666255
iteration 140: loss 0.08695416115831198


In [55]:
_, pred = model.forward(X_test.T)
acc = accuracy(y_test.reshape(1, -1), pred)

print(f'accuracy {acc}')

accuracy 0.9425


In [46]:
from model import deepNN as deepNNOld

In [51]:
layers = [200, 100, 20, 1]
model_unreg = deepNNOld(layers)

Actually when we have the `iteration` goes up, the model would continue to overfit that causes error in the divide operation, suspecting that in the forward process, result $A$ gets too close to 0.

In contrast, the model above with regularization would not overfit.

In [52]:
model_unreg.train(X_train, y_train, batch_size=200, n_iter=150, lr=0.05)

iteration 0: loss 0.6930918829042935
iteration 10: loss 0.6930065395149767
iteration 20: loss 0.6929575889989992
iteration 30: loss 0.6926539088979596
iteration 40: loss 0.6849650117201506
iteration 50: loss 0.20267451014178056
iteration 60: loss 0.09037465413243737
iteration 70: loss 0.04981389115902148
iteration 80: loss 0.02654689714177362
iteration 90: loss 0.015971046473694038
iteration 100: loss 0.010199685977249701
iteration 110: loss 0.007221608028851772
iteration 120: loss 0.004961759731198219
iteration 130: loss 0.0034589244309720397
iteration 140: loss 0.0025630729230403


In [53]:
_, pred = model_unreg.forward(X_test.T)
acc = accuracy(y_test.reshape(1, -1), pred)

print(f'accuracy {acc}')

accuracy 0.9335
